In [1]:
import gym 
import numpy as np
from custom_envs.gridworlds import WindyGridworldEnv, SimpleGridworldEnv
from custom_envs.mountain_car import MountainCarEnv

from function_approximators.function_approximators import NeuralNetwork, LinearModel, DecisionTree, RandomForest, SupportVectorRegressor, KNeighboursRegressor, GaussianProcess, OnlineGaussianProcess
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, RationalQuadratic, Matern
from sklearn.metrics.pairwise import rbf_kernel, chi2_kernel, laplacian_kernel 

from utils.train_utils import train, solve, train_time
from utils.plot_utils import plot_returns

from agents.av_agents import DQNAgent, LinearAgent, FQIAgent, eGaussianProccessAgent, OnlineGaussianProccessAgent

import operator


In [2]:
function_approximOnlineGaussianProcess]
agents = [DQNAgent, LinearAgent, *[FQIAgent]*5, OnlineGaussianProccessAgent]

RENDER = False
env = gym.make("CartPole-v1")
# env = gym.make("Acrobot-v1")
# env = gym.make("MountainCar-v0")
# env = WindyGridworldEnv()
# env = gym.make("LunarLander-v2")
# env = SimpleGridworldEnv()
# env = MountainCarEnv()

In [3]:
# DQN Config
CONFIG_DQN = {
    "episode_length": 200,
    "max_timesteps": 20000,
    "max_time": 30 * 60,
    "eval_freq": 1000, 
    "eval_episodes": 10,
    "learning_rate": 0.00075,
    "hidden_size": (32,32),
    "target_update_freq": 200,
    "batch_size": 32,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "plot_loss": False,
    "epsilon": 1,
    "max_deduct": 0.97,
    "decay": 0.25,
    "lr_step_size": 1000,
    "lr_gamma": 0.95,
    "max_steps": 200,
    "non_param": False,
}

# Linear Config
CONFIG_LINEAR = {
    "episode_length": 200,
    "max_timesteps": 20000,
    "max_time": 30 * 60,
    "eval_freq": 1000, 
    "eval_episodes": 10,
    "learning_rate": 0.02,
    "target_update_freq": 50,
    "batch_size": 32,
    "gamma": 0.99,
    "buffer_capacity": int(1e7),
    "plot_loss": False,
    "epsilon": 1,
    "max_steps": 200,
    "poly_degree": 2,
    "max_deduct": 0.97,
    "decay": 0.5,
    "lr_step_size": 1000,
    "lr_gamma": 0.99,
    "non_param": False,
}

# Decision Tree Config
CONFIG_DT = {
    "episode_length": 200,
    "max_timesteps": 20000,
    "max_time": 30 * 60,
    "eval_freq": 1000, 
    "eval_episodes": 10,
    "model_save_freq": 1000,
    "model_save_capacity": 20,
    "update_freq": 1,
    "batch_size": 512,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "epsilon": 1,
    "max_deduct": 0.95,
    "decay": 0.4,
    "max_steps": 200,
    "non_param": True,
    "model_params": {"criterion":"mse","max_depth": 10, "min_samples_split": 20, "min_samples_leaf": 5},
    "feature_names": ["Cart Position", "Cart Velocity", "Pole Angle", "Pole Angular Velocity", "Action: Push Left", "Action: Push Right"],
    "plot_name": "dt_depth=8",
}

# Random Forest Config
CONFIG_RF = {
    "episode_length": 200,
    "max_timesteps": 20000,
    "max_time": 30 * 60,
    "eval_freq": 1000, 
    "eval_episodes": 5,
    "model_save_freq": 1000,
    "model_save_capacity": 20,
    "update_freq": 5,
    "batch_size": 512,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "epsilon": 1,
    "max_deduct": 0.95,
    "decay": 0.4,
    "max_steps": 200,
    "non_param": True,
    "model_params": {"n_estimators": 10,"max_depth": 10, "min_samples_split": 20, "min_samples_leaf": 5},
}

# Support Vector Regressor Config
CONFIG_SVR = {
    "episode_length": 200,
    "max_timesteps": 20000,
    "max_time": 30 * 60,
    "eval_freq": 1000, 
    "eval_episodes": 10,
    "model_save_freq": 1000,
    "model_save_capacity": 20,
    "update_freq": 1,
    "batch_size": 512,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "epsilon": 1,
    "max_deduct": 0.95,
    "decay": 0.3,
    "max_steps": 200,
    "non_param": True,
    "model_params": {"kernel":"rbf", "degree": 2, "C": 2},
}


# K-Neighbors Regressor Config
CONFIG_KNR = {
    "episode_length": 200,
    "max_timesteps": 20000,
    "max_time": 30 * 60,
    "eval_freq": 1000, 
    "eval_episodes": 5,
    "model_save_freq": 1000,
    "model_save_capacity": 20,
    "update_freq": 1,
    "batch_size": 256,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "epsilon": 1,
    "max_deduct": 0.95,
    "decay": 0.3,
    "max_steps": 200,
    "non_param": True,
    "model_params": {"n_neighbors":7, "weights": "distance", "algorithm": "auto", "leaf_size": 30},
}

# Gaussian Process Config
CONFIG_GP = {
    "episode_length": 200,
    "max_timesteps": 20000,
    "max_time": 30 * 60,
    "eval_freq": 1000, 
    "eval_episodes": 5,
    "model_save_freq": 1000,
    "model_save_capacity": 20,
    "update_freq": 10,
    "batch_size": 512,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "epsilon": 1,
    "max_deduct": 0.95,
    "decay": 0.3,
    "max_steps": 200,
    "non_param": True,
    "model_params": {"alpha": 1e-10, "normalize_y": False, "kernel":  RBF(length_scale=0.08, length_scale_bounds="fixed")},
}

# Online Gaussian Process Config
CONFIG_GP_Online = {
    "episode_length": 200,
    "max_timesteps": 20000,
    "max_time": 30 * 60,
    "eval_freq": 1000, 
    "eval_episodes": 10,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "batch_size": 32,
    "epsilon": 1,
    "max_deduct": 0.95,
    "decay": 0.3,
    "max_steps": 200,
    "non_param": True,
    "model_params": {"sigma_0": 0.5, "kernel":  rbf_kernel, "epsilon_tol": 0.05, "basis_limit": 1000},
}

In [4]:
returns = []
train_returns = []
train_times = []
n_seeds=10

j=10
for i in range(n_seeds):
    print(f"\n Run: {i+1} \n")
    r, _, t, times = train(env, 
            CONFIG_GP_Online, 
            fa=function_approximators[j], 
            agent = agents[j], 
            render=RENDER,
            online=True)
    env.close()
    returns.append(r)
    train_returns.append(t)
    train_times.append(times)
    


  0%|          | 67/20000 [00:00<00:40, 489.17it/s]
 Run: 1 

  5%|▌         | 1004/20000 [00:10<04:49, 65.69it/s]Evaluation at timestep 1004 returned a mean returns of 200.0
Epsilon = 0.8510083333333334
Support Points = 825
 10%|█         | 2003/20000 [00:22<04:12, 71.24it/s]Evaluation at timestep 2003 returned a mean returns of 114.1
Epsilon = 0.68745
Support Points = 1175
 15%|█▌        | 3003/20000 [00:37<03:29, 81.29it/s]Evaluation at timestep 3003 returned a mean returns of 178.60000000000002
Epsilon = 0.530225
Support Points = 1381
 20%|██        | 4020/20000 [00:59<05:43, 46.51it/s]Evaluation at timestep 4020 returned a mean returns of 119.8
Epsilon = 0.38250000000000006
Support Points = 1673
 25%|██▌       | 5078/20000 [01:30<06:37, 37.57it/s]Evaluation at timestep 5078 returned a mean returns of 167.9
Epsilon = 0.22163333333333335
Support Points = 1986
 31%|███       | 6146/20000 [02:05<07:05, 32.57it/s]Evaluation at timestep 6146 returned a mean returns of 196.9
Epsilon = 0.

KeyboardInterrupt: 

In [5]:
with open(f'cartpole_eval_Gaussian Process (On+).csv', 'ab') as eval:
    for i in range(n_seeds):
        np.savetxt(eval, [returns[i]], delimiter=',')

IndexError: list index out of range

In [6]:
with open(f'cartpole_train_Gaussian Process (On+).csv', 'ab') as train:
    for i in range(n_seeds):
        np.savetxt(train, [train_returns[i]], delimiter=',')
        np.savetxt(train, [train_times[i]], delimiter=',')

IndexError: list index out of range

In [10]:
n_eps = []
n_steps = []
not_solved = []
n_seeds=30

j=8
for i in range(n_seeds):
    print(f"\n Run: {i+1} \n")
    s, e, n = solve(env, 
            CONFIG_GP, 
            fa=function_approximators[j], 
            agent = agents[j],
            target_return=195,
            op=operator.ge, 
            render=RENDER)
    env.close()
    n_eps.append(e)
    n_steps.append(s)
    not_solved.append(n)


 Run: 1 

Ep. timesteps: 200
Total timesteps: 985
Total episodes: 36
Evaluation mean return: 200.0

 Run: 2 

Ep. timesteps: 200
Total timesteps: 1873
Total episodes: 80
Evaluation mean return: 196.2

 Run: 3 

Ep. timesteps: 200
Total timesteps: 1517
Total episodes: 61
Evaluation mean return: 200.0

 Run: 4 

Ep. timesteps: 200
Total timesteps: 1631
Total episodes: 60
Evaluation mean return: 200.0

 Run: 5 

Ep. timesteps: 200
Total timesteps: 2103
Total episodes: 78
Evaluation mean return: 200.0

 Run: 6 

Ep. timesteps: 195
Total timesteps: 1048
Total episodes: 41
Evaluation mean return: 199.0

 Run: 7 

Ep. timesteps: 200
Total timesteps: 1004
Total episodes: 41
Evaluation mean return: 200.0

 Run: 8 

Ep. timesteps: 200
Total timesteps: 1388
Total episodes: 65
Evaluation mean return: 195.8

 Run: 9 

Ep. timesteps: 200
Total timesteps: 4551
Total episodes: 116
Evaluation mean return: 200.0

 Run: 10 

Ep. timesteps: 200
Total timesteps: 857
Total episodes: 37
Evaluation mean retu

In [11]:
with open(f'cartpole_se195_Gaussian Process.csv', 'ab') as se:
    np.savetxt(se, [n_eps], delimiter=',')
    np.savetxt(se, [n_steps], delimiter=',')
    np.savetxt(se, [not_solved], delimiter=',')

In [12]:
mean_eps = np.mean(n_eps)
std_eps = np.std(n_eps)
print(f"Average n_eps: {mean_eps}")
print(f"Std n_eps: {std_eps}")
print(f"St.error n_eps: {std_eps/np.sqrt(n_seeds)}")

mean_steps = np.mean(n_steps)
std_steps = np.std(n_steps)
print(f"Average n_steps: {mean_steps}0")
print(f"Std n_steps: {std_steps}")
print(f"St.error n_steps: {std_steps/np.sqrt(n_seeds)}")

print(f"Not solved: {np.sum(not_solved)} runs")

Average n_eps: 77.13333333333334
Std n_eps: 47.77219088781906
St.error n_eps: 8.721968856900416
Average n_steps: 3211.03333333333330
Std n_steps: 4515.506184126967
St.error n_steps: 824.4148652001386
Not solved: 1 runs


In [14]:
j = 10
time = train_time(env, 
        CONFIG_GP_Online, 
        fa=function_approximators[j], 
        agent = agents[j],
        online=True)
env.close()

print(time)

20029it [06:32, 51.03it/s]200.0
392.48629212379456

